In [1]:
import modelskill as ms
import xarray as xr
import pandas as pd
import numpy as np

(The following data structure can be automatically generated using a `NetworkMapper` from `mikeio1d`)

In [2]:
ds = xr.load_dataset("../tests/testdata/network.nc")
ds

<xarray.Dataset> Size: 231kB
Dimensions:     (time: 110, node: 259)
Coordinates:
  * time        (time) datetime64[ns] 880B 1994-08-07T16:35:00 ... 1994-08-07...
  * node        (node) int64 2kB 0 1 2 3 4 5 6 7 ... 252 253 254 255 256 257 258
Data variables:
    Discharge   (time, node) float32 114kB nan nan 5.72e-06 ... nan 0.01692 0.0
    WaterLevel  (time, node) float32 114kB 195.4 194.7 nan ... 188.5 nan nan

# Create Network Model Result

Now we'll create a `NetworkModelResult` from the network dataset.

In [3]:
network_model = ms.NetworkModelResult(ds, item="WaterLevel")
network_model

<NetworkModelResult>: WaterLevel

# Extract Model Data at Observation Nodes

Now we'll extract the network model data at each observation location.

In [4]:
# Script to generate dummy sensor data

sensor_1 = ds["WaterLevel"].sel(node=30).to_pandas().rename("water_level@sens1")
sensor_2 = ds["WaterLevel"].sel(node=54).to_pandas().rename("water_level@sens2")
sensor_3 = ds["WaterLevel"].sel(node=71).to_pandas().rename("water_level@sens3")

for n, sensor in enumerate([sensor_1, sensor_2, sensor_3], start=1):
    sensor += np.random.normal(0, sensor.mean() / 2, len(sensor))
    sensor.index = [sensor.index[i] + pd.Timedelta(s, unit="s") for i, s in enumerate(np.random.uniform(-10, 10, len(sensor)))]
    sensor.sort_index(inplace=True)
    if n == 2:
        sensor = sensor.iloc[30:]
    if n == 3:
        sensor = pd.concat([sensor.iloc[:50], sensor.iloc[70:]])

    sensor.to_csv(f"../tests/testdata/network_sensor_{n}.csv")

In [5]:
# The name is taken from the name of the series
single_obs = ms.NodeObservation(sensor_2, node=54)
ms.match(single_obs, network_model).skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
water_level@sens2,108,3.733807,96.258762,96.186319,76.559918,-0.090601,0.500792,-0.001848


In [6]:
# The name is taken from the name of the file
# - The numbers change because the csv contains a subset of the series
path_to_sensor2 = "../tests/testdata/network_sensor_2.csv"
single_obs = ms.NodeObservation(path_to_sensor2, node=54)
ms.match(single_obs, network_model).skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
network_sensor_2,79,9.202745,101.176901,100.757505,79.09441,-0.046308,0.537481,-0.008495


In [7]:
# The name is passed
single_obs = ms.NodeObservation(path_to_sensor2, node=54, name="Sensor 2")
ms.match(single_obs, network_model).skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
Sensor 2,79,9.202745,101.176901,100.757505,79.09441,-0.046308,0.537481,-0.008495


## Multiple sensors

In [8]:
sensor_df = pd.concat([sensor_1, sensor_2, sensor_3], axis=1)

In [9]:
multi_obs = ms.NodeObservation.from_multiple(data=sensor_df, nodes=[30, 54, 71])
ms.match(multi_obs, network_model).skill()

C:\Users\japr\AppData\Local\Temp\ipykernel_26200\2937751291.py:1: UserWarning: 'nodes' was passed as a list of length 3 so, only the first 3 items of 'data' will be selected to match the nodes. You can pass 'nodes' as a dictionary to assign an item to each node.
  multi_obs = ms.NodeObservation.from_multiple(data=sensor_df, nodes=[30, 54, 71])


,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
water_level@sens1,109,-9.313641,108.977163,108.578442,87.977385,-0.027785,0.527225,-0.007597
water_level@sens2,108,3.733807,96.258762,96.186319,76.559918,-0.090601,0.500792,-0.001848
water_level@sens3,109,-3.543909,98.192695,98.128721,78.211039,0.025243,0.492774,-0.001037


In [10]:
# exactly the same, without warning
multi_obs = ms.NodeObservation.from_multiple(data=sensor_df, nodes={30: "water_level@sens1", 54: "water_level@sens2", 71: "water_level@sens3"})
ms.match(multi_obs, network_model).skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
water_level@sens1,109,-9.313641,108.977163,108.578442,87.977385,-0.027785,0.527225,-0.007597
water_level@sens2,108,3.733807,96.258762,96.186319,76.559918,-0.090601,0.500792,-0.001848
water_level@sens3,109,-3.543909,98.192695,98.128721,78.211039,0.025243,0.492774,-0.001037


In [11]:
# pd.Series have a unique item name 
multi_obs = ms.NodeObservation.from_multiple(nodes={30: sensor_1, 54: path_to_sensor2, 71: sensor_3})
ms.match(multi_obs, network_model).skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
water_level@sens1,109,-9.313641,108.977163,108.578442,87.977385,-0.027785,0.527225,-0.007597
network_sensor_2,79,9.202745,101.176901,100.757505,79.094410,-0.046308,0.537481,-0.008495
water_level@sens3,109,-3.543909,98.192695,98.128721,78.211039,0.025243,0.492774,-0.001037
